# Creating a GPT From Scratch

Workbook will explain every aspect of the code. Process:

- __Setup__:Convert the words into simple tokens
- Feed these tokens into an embedding layer, which will be a randomised vector representation of 364. The index position of these embeddings will relate to the tokens.
- Run this through multiple heads of self attention
- 

## Setup

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [2]:
# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2
# ------------

In [3]:

torch.manual_seed(1337)

# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('../data/input.txt', 'r', encoding='utf-8') as f:
    text = f.read()


Create the training and test data.

In [4]:

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

train_data

tensor([18, 47, 56,  ..., 43, 56, 43])

Functions to create the batch and blocks for the model:

- Batch size (B) is the number of training samples processed for each iteration of updating the model.
- Block size (T) is the context length, how many tokens are processed in a sequence at once.

The loss is calcualted using cross-entropy, this is a measure of how well the model did to predict the next letter. It is essentially 

$$
\text{Total Loss} = \frac{1}{N} \sum_{o=1}^N \left( -\sum_{c=1}^M y_{o,c} \log(p_{o,c}) \right)$$


Where:
- **\( L \)**: The loss for a single observation.
- **\( M \)**: The total number of classes.
- **\( y_{o,c} \)**: A binary indicator (0 or 1) that is 1 if class label \( c \) is the correct classification for observation \( o \), and 0 otherwise.
- **\( p_{o,c} \)**: The predicted probability of observation \( o \) being of class \( c \). This probability is output from the model, typically from a softmax layer that normalizes the logits output by the neural network.
$$


In [5]:

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In this case the block size is 265 and the block size is 64.

In [6]:
# sample a batch of data
xb, yb = get_batch('train')

print(xb.shape)
print(yb.shape)

torch.Size([64, 256])
torch.Size([64, 256])


## Model

### Self Attention Layer

This is an attention layer. To understand the mathematics of what is happening here please go to: 

https://github.com/jotren/Machine-Learning-Teaching-Material/blob/main/theory/Transformer%20Encoder%20Maths.md

In [7]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out


### Multi Head Attention

We need to employ multi head attention. Each head in the multi-head attention mechanism can potentially focus on different aspects of the input data, leading to a rich spectrum of learned relationships. Here’s a deeper look at how this diversity across different heads adds value.

In [8]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

This is a standard feed forward neural network

In [9]:
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)


This block contains multihead attention with a feedfoward and some normalisation. Normlisation layers are important to help with:

- Training Stability
- Accelrates Convergance
- Robust with respect to initial weights

In [10]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x


### GPT model

This model brings everything together of initialising weights, trainin

In [11]:
class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        # better init, not covered in the original GPT video, but important, will cover in followup video
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx
    
    def save(self, path):
        """Saves the model's state_dict (weights) to a file."""
        torch.save(self.state_dict(), path)

    def load(self, path, device='cpu'):
        """Loads the model's weights from a file."""
        # Loads the saved state_dict into the model.
        # This method also ensures that the model is loaded for the correct device.
        self.load_state_dict(torch.load(path, map_location=device))
        self.eval()  # Set the model to evaluation mode


### Training

First we need to set up our model. Will use the ADAM optimiser.

In [12]:
model = GPTLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

10.788929 M parameters


Then we loop over the optimiser, at the start of each loop we evaluate the performance with evaluate_loss(). Note: this already has the model loaded as it is a global variable.

Then we get a batch from training data of x and y, we get the resulting logits from the model 

In [13]:
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 4.2209, val loss 4.2191
step 500: train loss 1.7476, val loss 1.8964
step 1000: train loss 1.3990, val loss 1.5953
step 1500: train loss 1.2706, val loss 1.5244
step 2000: train loss 1.1882, val loss 1.4994
step 2500: train loss 1.1282, val loss 1.4953
step 3000: train loss 1.0731, val loss 1.4815
step 3500: train loss 1.0223, val loss 1.5040
step 4000: train loss 0.9631, val loss 1.5112
step 4500: train loss 0.9133, val loss 1.5411
step 4999: train loss 0.8666, val loss 1.5700


### Model Ouput

In [14]:
model.save('../models/model_weights.pth')

In [18]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))
open('../result/generated_text.txt', 'w').write(decode(m.generate(context, max_new_tokens=10000)[0].tolist()))



BUSHY:
Then in the conditions repolies of diseal peers.

NORTHUMERS:
The citizens are they they comes to Mominous.

THOMAS MOWBRAY:
Why, there have I lived your former stair-speaking,
Not humbled?

KING RICHARD III:
It came, my lady; I to his castles,
Looks who do' secret your spring.

DUKE OF AUMERLE:
My Lancaster, and lord Base
Shall be appared at nympt. This once I would out him;
I am a slain a brace of the all-babe't dead,
Provided violence to in the unlook on.

ERCUTIO:
Fare you all patien


10001